In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
from model import ResNetCIFAR
from data_loader import get_loader
import math
import sys
import os

50000it [00:03, 14872.57it/s]
50000it [00:03, 15358.19it/s]
50000it [00:03, 15359.60it/s]


In [2]:
batch_size = 128
num_epochs = 5
save_every = 1
print_every = 100
log_file = 'training_log.txt'

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

train_data_loader = get_loader(transform = transform_train,
                              mode = 'train', batch_size = batch_size, data_folder = './data')

50000it [00:03, 15663.94it/s]


In [3]:
resnet_fine = ResNetCIFAR(image_channels = 3, num_classes = 100, expansion = 1, num_blocks_per_layer = 2)
resnet_coarse = ResNetCIFAR(image_channels = 3, num_classes = 20, expansion = 1, num_blocks_per_layer = 2)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_fine.to(device)
resnet_coarse.to(device)

criterion_fine = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()
criterion_coarse = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()


optimizer_fine = optim.Adam(resnet_fine.parameters(), lr = 0.001)
optimizer_coarse = optim.Adam(resnet_coarse.parameters(), lr = 0.001)

total_step = math.ceil(len(train_data_loader.dataset.cifar.file_names) / train_data_loader.batch_sampler.batch_size)

In [5]:
import torch.utils.data as data
import numpy as np
import requests
import time

In [11]:
f = open(log_file, 'w')

start_time = time.time()

for epoch in range(1, num_epochs + 1):
    for i_step in range(1, total_step + 1):
        # Obtain the batch
        images, fine_labels, coarse_labels = next(iter(train_data_loader))
        
        # move the batch to GPU if cuda if available
        images, fine_labels, coarse_labels = images.to(device), fine_labels.to(device), coarse_labels.to(device)
        
        # zero the gradients
        resnet_fine.zero_grad()
        resnet_coarse.zero_grad()
        
        # pass the input through the ResNet model
        output_fine = resnet_fine(images)
        output_coarse = resnet_coarse(images)
        
        # Calculate the batch loss
        loss_fine = criterion_fine(output_fine, fine_labels)
        loss_coarse = criterion_coarse(output_coarse, coarse_labels)
        
        # Backward pass
        loss_fine.backward()
        loss_coarse.backward()
        
        # Update the parameters in the optimizer
        optimizer_fine.step()
        optimizer_coarse.step()
        
        # get training statistics
        stats_fine = 'Epoch [%d/%d], Step [%d/%d], Fine Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, 
                                                                                     total_step, loss_fine.item(),
                                                                                    np.exp(loss_fine.item()))
        stats_coarse = 'Epoch [%d/%d], Step [%d/%d], Coarse Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, 
                                                                                     total_step, loss_coarse.item(),
                                                                                    np.exp(loss_coarse.item()))
        
        # Print training statistics (on same line)
        print('\r' + stats_fine, end = "")
        print('\r' + stats_coarse, end = "")
        sys.stdout.flush()
        
        # Print training statistics to file
        f.write(stats_fine + '\n')
        f.write(stats_coarse + '\n')
        f.flush()
        
        # Print training statistics (every print_every lines)
        if i_step % print_every == 0:
            print('\r' + stats_fine)
            print('\r' + stats_coarse)

            
    if not os.path.exists(os.path.join(os.getcwd(), 'models')):
        os.mkdir(os.path.join(os.getcwd(), 'models'))
        
    if epoch % save_every == 0:
        torch.save(resnet_fine.state_dict(), os.path.join(os.getcwd(), 'models/resnet_fine-%d.pkl' % epoch))
        torch.save(resnet_coarse.state_dict(), os.path.join(os.getcwd(), 'models/resnet_coarse-%d.pkl' % epoch))
f.close()

Epoch [1/3], Step [100/391], Fine Loss: 0.0011, Perplexity: 1.001105
Epoch [1/3], Step [100/391], Coarse Loss: 0.0005, Perplexity: 1.0005
Epoch [1/3], Step [200/391], Fine Loss: 0.0009, Perplexity: 1.000904
Epoch [1/3], Step [200/391], Coarse Loss: 0.0004, Perplexity: 1.0004
Epoch [1/3], Step [300/391], Fine Loss: 0.0008, Perplexity: 1.000803
Epoch [1/3], Step [300/391], Coarse Loss: 0.0003, Perplexity: 1.0003
Epoch [2/3], Step [100/391], Fine Loss: 0.0006, Perplexity: 1.000602
Epoch [2/3], Step [100/391], Coarse Loss: 0.0002, Perplexity: 1.0002
Epoch [2/3], Step [200/391], Fine Loss: 0.0005, Perplexity: 1.000502
Epoch [2/3], Step [200/391], Coarse Loss: 0.0002, Perplexity: 1.0002
Epoch [2/3], Step [300/391], Fine Loss: 0.0004, Perplexity: 1.000402
Epoch [2/3], Step [300/391], Coarse Loss: 0.0002, Perplexity: 1.0002
Epoch [3/3], Step [100/391], Fine Loss: 0.0003, Perplexity: 1.000301
Epoch [3/3], Step [100/391], Coarse Loss: 0.0001, Perplexity: 1.0001
Epoch [3/3], Step [200/391], Fine 

In [52]:
print('\r')

 asdf
